### RAG inferencing pipeline

Types of user queries to handle with agentic retrieval:
1. No question asked -> LLM response
2. Single and simple question about data -> Rewrite -> Embed -> Rerank -> LLM response
3. Complex question about data -> Expand question into sub questions -> Embed -> Reciprocal rank fusion -> LLM response
4. Multiple questions about data -> Sub-query decomposition -> Embed -> Rerank -> LLM response

LLM prompts:
- Classification
- Query rewriting
- Query rewriting + query expansion
- Query rewriting + sub-query decomposition
- Reranking
- Reciprocal Rank Fusion
- Regular response generation
- Response generation with retrieved documents

In [6]:
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
from pydantic import BaseModel, conint, conlist 
import json
from operator import itemgetter
import time
import pandas as pd

In [7]:
#Variables
service_endpoint = ""
# index_name = "idx-exhaustiveknn-openai-vectorizer-text-embedding-3-large"
index_name = ""
key = ""

#max number of sub queries that can be generated from a single complex user query
max_num_queries_complex = 3

#max number of sub queries that can be generated from a single user query that contains multiple questions
max_num_queries_multiple = 5

#Number of relevant documents being returned by the search algorithm
nr_search = 100

#Number of relevant documents the reranker returns to the LLM
nr_docs = 20

#Number of relevant documents the RRF algorithm returns to the LLM
nr_rrf_docs = 20


#Configuration of retrieval methods
#Each tuple contains (method_name, use_agentic, use_reranking)
selected_methods = [
    ("Non-agentic retrieval", False, False),
    ("Agentic retrieval without reranking", True, False),
    ("Agentic retrieval with reranking", True, True)
]

#List of all index names from Azure AI search to use
all_index_names = [
    "idx-exhaustiveknn-custom-vectorizer-384",
    "idx-exhaustiveknn-openai-vectorizer-text-embedding-3-large",
    "idx-exhaustiveknn-openai-vectorizer-text-embedding-ada-002",
    "idx-hnsw-custom-vectorizer-384",
    "idx-hnsw-openai-vectorizer-text-embedding-3-large",
    "idx-hnsw-openai-vectorizer-text-embedding-ada-002"
]



In [8]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "")

#Model selection
client = AzureOpenAI(
     api_version="2024-12-01-preview",
     azure_endpoint="",
     azure_ad_token_provider=token_provider
 )

deployment_names = ["gpt-4o", "gpt-4.1"]

#Set up the AI Search client
search_client = SearchClient(
     endpoint=service_endpoint,
     index_name=index_name,
     credential=AzureKeyCredential(key)
 )


In [10]:
CLASSIFICATION_PROMPT="""
You are an AI assistant that help users retrieve information from the source material.
The source material for answering the questions is the "Handboek Financiële Informatie en Administratie Rijksoverheid" manual, which is in Dutch. Below is a description of its content:
###
De verschillende bronnen beschrijven *diverse aspecten van het Nederlandse overheidsbeleid en financieel beheer, waaronder **begrotingsprocessen, de **rol van verschillende overheidsinstanties* en *specifieke richtlijnen voor subsidieverlening en projectmanagement. Ze behandelen de **structuur en verantwoording van overheidsfinanciën, de **evaluatie van beleid op doeltreffendheid en doelmatigheid, en **protocollen voor grote projecten en het schatkistbankieren. De teksten belichten ook de **principes van duurzame ontwikkeling en brede welvaart* in relatie tot beleidsvorming, evenals *juridische kaders en beginselen* die de uitvoering van overheidsbeleid sturen. Tenslotte wordt aandacht besteed aan de *communicatie van overheidsinformatie* en de *betrokkenheid van belanghebbenden* bij beleidsprocessen.
###

Instructions: 
###
You are given the user query in Dutch.
This query should either be used for document retrieval, or it should warrent a regular response without retrieval.
Your task is to classify the user query into one of the four categories. 
You are responsible for selecting the correct category such that a specialized downstream task can be applied correctly.
Return ONLY the number 1, 2, 3, or 4, that corresponds to the correct category.
Do not include any other details in your response, only provide a single number.

Choose which of the four category the query belongs to:
1: The query is completely unrelated to "Het Handboek Financiële Informatie en Administratie Rijksoverheid"
2: The query contains a single simple question that can directly be answered through embedding and document search.
3: the query contains a single complex question that warrents the query to be expanded first, before applying embedding and document search.
4. The query contains multiple questions that need to be split up such that they can be embedded and answered separately. 

Examples:
---
Example_query_0: "Goedendag"
Reponse: 1

Example_query_1: "Welke kleuren heeft de regenboog?"
Response: 1

Example_query_2: "Welke departementen bevinden zich binnen het Ministerie van Financiën?"
Reponse: 2

Example_query_3: "Hebben jullie een procesbeschrijving van de controle van begrotingswetten en publicatie in de Staatscourant?
Response: 3

Example_query_4: "Hoe kan de overheid kosten besparen, en welke praktische stappen zijn nodig om dit te bereiken? Hoe wordt de rol van inspecteurs in dit proces bepaald?"
Reponse: 4
---
###

User query:
### 
{query} 
###

Response:
"""

In [11]:
#Structured LLM output for query classification
#Restricts the output to be an int between 1 and 4
class ClassificationEvent(BaseModel):
    classification: conint(ge=1, le=4)

def classify_query(query, deployment_name):
    """
    Classifies the user query using the classification prompt.
    Returns thee classification result for the query.
    """
    query_classification_response = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": CLASSIFICATION_PROMPT.format(query=query)
            }
        ],
        model=deployment_name,
        response_format=ClassificationEvent
    )
    
    # Extract the classification from the response
    classification = json.loads(query_classification_response.choices[0].message.content)["classification"]
    return classification

In [13]:
CONVERSATIONAL_PROMPT="""
You are a helpful AI assistant. Assist the user with any questions or tasks to the best of your ability.

User query:
###
{query}
###
"""

In [14]:
SIMPLE_PROMPT="""
You are an expert in refining a simple user question into a clear and searchable query. 

Instructions:  
* Analyze the user's query and identify ways to improve clarity and flow while preserving its meaning.  
* Do NOT add new information or alter the query's original intent.  
* If the query contains ambiguous wording, aim to structure it more clearly and with greater precision.  
* Maintain the tone and style of the original question.
* If there are acronyms or words you are not familiar with, do not rephrase them.
* Generate the query in the same language as the orignal user query.

Here are some examples:  
###  
Example_user_query: "Hoe omgaan belastingplichtigen die typisch met bijzondere situaties?"  
Query: "Hoe gaan belastingplichtigen om met bijzondere situaties?"  

Example_user_query: "Welk stappen moet ik doorlopen voor overheid financien precies gecontroleed?"  
Query: "Welke stappen moeten worden doorlopen om overheidsfinanciën precies te controleren?"  
###  

User query:
### 
{query} 
###

Query:

"""

In [15]:
#Structured LLM output for query rewriting
#Constraints the output to be a single string
class SimpleQueryEvent(BaseModel):
    queries: str


In [16]:
COMPLEX_PROMPT="""
You are an expert in expanding complex user questions into actionable sub-queries. Your expertise lies in analyzing complex questions, breaking them down into multiple smaller sub-queries that together answer the original question effectively. 

The source material for answering the questions is the "Handboek Financiële Informatie en Administratie Rijksoverheid" manual, which is in Dutch. Below is a description of its content:
###
De verschillende bronnen beschrijven *diverse aspecten van het Nederlandse overheidsbeleid en financieel beheer, waaronder **begrotingsprocessen, de **rol van verschillende overheidsinstanties* en *specifieke richtlijnen voor subsidieverlening en projectmanagement. Ze behandelen de **structuur en verantwoording van overheidsfinanciën, de **evaluatie van beleid op doeltreffendheid en doelmatigheid, en **protocollen voor grote projecten en het schatkistbankieren. De teksten belichten ook de **principes van duurzame ontwikkeling en brede welvaart* in relatie tot beleidsvorming, evenals *juridische kaders en beginselen* die de uitvoering van overheidsbeleid sturen. Tenslotte wordt aandacht besteed aan de *communicatie van overheidsinformatie* en de *betrokkenheid van belanghebbenden* bij beleidsprocessen.
###

Instructions:
* Analyze the complex user query and understand its core components.
* These sub-queries, when answered, should collectively help address the original user query.
* Generate a maximum of {max_num_queries_complex} sub-queries, one on each line. 
* Do not include any other details in your response, only provide the queries.

These are the guidelines you consider when completing your task:
* Carefully analyze the user query and understand the core question.
* The sub queries should be relevant to the user query.
* Identify different aspects, contexts, or components required to answer the original query.
* Generate clear, specific, and actionable sub-queries that will assist in gathering relevant information for answering the original query.
* If there are acronyms or words you are not familiar with, do not try to rephrase them.
* Generate the sub-queries in the same language as the orignal user query.

Example: 
###
Example_user_query: "Hoe worden overheidsfinanciën gecontroleerd en beheerd, inclusief budgettering en verantwoording?"
Queries: Hoe worden overheidsfinanciën beheerd volgens comptabele regelgeving?  
Welke voorschriften gelden binnen de rijksoverheid voor budgettering en financieel beheer?  
Wat zijn de regels en procedures voor financiële controle en verantwoording binnen de rijksoverheid?  
###

User query:
### 
{query} 
###

Queries:

"""

In [17]:
#Structured LLM output for query expansion
#Constraints the output to be a list of strings, ranging from 1 to n objects
class ComplexQueryEvent(BaseModel):
    queries: conlist(str, min_length = 1, max_length = max_num_queries_complex)


In [18]:
MULTIPLE_PROMPT="""
You are an expert in analyzing and organizing user queries that contain multiple questions into separate questions.
You will receive a user query that contains multiple questions.
Your task is to separate the user query into distinct and clear individual queries that can be used for document search.

Instructions:  
* Carefully analyze the user query and identify each distinct question it contains.  
* Decompose the query into separate, logically ordered queries, ensuring each one is complete and actionable on its own.  
* Maintain the original meaning and intent of each question.
* Only generate as many questions as the original user query contains.
* Do not include any other details in your response, only provide the queries.
* Generate a maximum of {max_num_queries_multiple} queries, one on each line.

These are the guidelines you consider when completing your task:
* Write each query clearly and concisely, one per line.
* Generate clear, specific, and actionable sub-queries that will assist in gathering relevant information for answering the original query.
* You may fix typos and improve clarity.
* If there are acronyms or words you are not familiar with, do not try to rephrase them.
* Ensure no question is omitted or altered in meaning.
* Generate the queries in the same language as the orignal user query.

Example: 
###
Example_query_1: "Hoe worden overheidsfinanciën gecontroleerd en beheerd? Hoe verschilt dit van budgettering in private organisaties?"
Queries: Hoe worden overheidsfinanciën gecontroleerd?  
Hoe worden overheidsfinanciën beheerd?  
Hoe verschilt budgettering in private organisaties van budgettering binnen de overheid? 
###

Query:
### 
{query}
###

Queries:

"""

In [20]:
#Structured LLM output for query decomposition
#Constraints the output to be a list of strings, ranging from 1 to n objects
class MultipleQueryEvent(BaseModel):
    queries: conlist(str, min_length = 1, max_length = max_num_queries_multiple)



In [21]:
grounded_prompt_rewriting="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
You will be given the original query from the user, as well as the transformed query that is used for optimized searching.
Answer with the facts listed in the list of sources below. 
Cite your source when you answer the question.
If there isn't enough information below, say that the sources don't provide an answer to the question.
Answer the user in Dutch.

Original query: 
###
{original_query}
###

Transformed query: 
###
{transformed_query}
###

Sources:
###
{sources}
###

"""

In [22]:
grounded_prompt_expansion="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
You will be given the original query from the user, as well as the expanded sub queries that were used for optimized searching.
Answer with the facts listed in the list of sources below. 
Cite your source when you answer the question.
If there isn't enough information below, say that the sources don't provide an answer to the question.
Answer the user in Dutch.

Original query: 
###
{original_query}
###

Expanded queries: 
###
{transformed_query}
###

Sources:
###
{sources}
###

"""

In [23]:
grounded_prompt_decomposition="""
You are an AI assistant that helps users learn from the information found in the source material.
The user query contains multiple questions.
An LLM has decomposed the query, extracting each question separately for optimized searching.
You will answer each query using their corresponding sources
Answer each query using the corresponding sources.
Answer with the facts listed in the list of sources below. 
Cite your source when you answer the question.
If there isn't enough information below, say that the sources don't provide an answer to the question.
Answer the user in Dutch.

Original query: 
###
{original_query}
###

Decomposed queries: 
###
{transformed_query}
###

Sources:
###
{sources}
###

"""

In [24]:
grounded_prompt_non_agentic="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
You will be given the original query from the user, as well as the transformed query that is used for optimized searching.
Answer with the facts listed in the list of sources below. 
Cite your source when you answer the question.
If there isn't enough information below, say that the sources don't provide an answer to the question.
Answer the user in Dutch.

Original query: 
###
{original_query}
###

Sources:
###
{sources}
###

"""

In [25]:
from FlagEmbedding import FlagReranker

#Loads reranker into memory, takes +- 2 minutes
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-08 21:56:10.015362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752011770.779153   30349 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752011770.996701   30349 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752011773.019012   30349 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than 

### Search Client invoke functions

In [31]:
class PipelineConfig:
    """
    Configuration for the retrieval pipeline.
    """
    def __init__(self, use_agentic_retrieval: bool = True, use_reranking: bool = True):
        self.use_agentic_retrieval = use_agentic_retrieval
        self.use_reranking = use_reranking

In [32]:
def process_search_results(search_results):
    '''
    Parses search_results information for each subquery for RRF
    '''
    chunk_dict = {}
    result_dict = {}
    for query, query_result in search_results.items():
        temp_dict = {}
        # print(query)
        for result in query_result:
            chunk_dict[result['chunk_id']] = result
            temp_dict[result['chunk_id']] = result['@search.score']
        result_dict[query] = temp_dict
    #result_dict contains the query as keys with chunk_id and score as values
    #chunk_dict contains all the info
    return result_dict, chunk_dict



#RRF implementation from https://github.com/Raudaschl/rag-fusion/blob/master/main.py
def reciprocal_rank_fusion(search_results_dict, k=60):
    fused_scores = {}
    for query, doc_scores in search_results_dict.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

In [33]:
def search_conversational(query, deployment_name):
    '''
    Directly generate a response to the user query without retrieval
    '''
    query_conversational_response = client.beta.chat.completions.parse(
    messages=[
        {
            "role": "user",
            "content": CONVERSATIONAL_PROMPT.format(query=query)
        }
    ],
    model=deployment_name,
    )
    return query_conversational_response.choices[0].message.content


def search_simple_with_optional_ranking(query, rerank, search_client, deployment_name):
    '''
    Execute simple search with query rewriting and reranking
    '''
    query_simple_response = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": SIMPLE_PROMPT.format(query=query)
            }
        ],
        model=deployment_name,
        response_format=SimpleQueryEvent
    )
    #Rewritten user query
    query_rewriting = json.loads(query_simple_response.choices[0].message.content)["queries"]

    #Configure vectorized query
    vector_query = VectorizableTextQuery(text=query_rewriting, k_nearest_neighbors=nr_search, fields="text_vector")

    #Perform vector search on the single query
    #Search_results is an iterator object containing the relevant documents
    #Convert result to list to prevent exhausting the iterator
    search_results = list(search_client.search(
        search_text=query_rewriting,
        vector_queries=[vector_query],
        select=["title", "chunk"],
        top=nr_search,
    ))

    #Use reranking or not based on configurations
    if rerank:
        #Use reranker to rerank the search results
        reranker_input = [[query_rewriting, document['chunk']] for document in search_results]
        scores = reranker.compute_score(reranker_input, normalize=True)

        ranked_documents = [
            {
                "title": document["title"],
                "chunk": document["chunk"],
                "score": score,
            }
            for document, score in zip(search_results, scores)
        ]

        #Sort documents by highest relevancy score and return top nr_docs documents
        top_documents = sorted(ranked_documents, key=itemgetter("score"), reverse=True)[:nr_docs]
    else:
        #Directly return the top nr_docs documents without reranking
        top_documents = search_results[:nr_docs]


    #Extract chunks of the top_documents to export to df and use for evaluation
    top_chunks = [doc["chunk"] for doc in top_documents]
    #print(top_chunks)

    #Format documents for LLM readability
    sources_formatted_simple = "=================\n".join(
        [f'TITLE: {doc["title"]}, CONTENT: {doc["chunk"]}' for doc in top_documents]
    )

    #Generate LLM response using the relevant documents
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": grounded_prompt_rewriting.format(
                    original_query=query,
                    transformed_query=query_rewriting,
                    sources=sources_formatted_simple,
                )
            }
        ],
        model=deployment_name
    )
    return response.choices[0].message.content, top_chunks


def search_complex_with_optional_ranking(query, rerank, search_client, deployment_name):
    '''
    Execute complex search with query expansion and (optional) reciprocal rank fusion (RRF)
    '''
    query_complex_response = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": COMPLEX_PROMPT.format(query=query, max_num_queries_complex=max_num_queries_complex)
            }
        ],
        model=deployment_name,
        response_format=ComplexQueryEvent
    )

    #Contains list of subqueries expanded from the original user query
    query_expansion = json.loads(query_complex_response.choices[0].message.content)["queries"]

    #print(f"query_expansion: {query_expansion}")
    search_results = {}
    
    #Perform vector search separately for each subquery
    #Store all results for all subqueries in search_results
    for subquery in query_expansion:
        vector_query = VectorizableTextQuery(text=subquery, k_nearest_neighbors=nr_search, fields="text_vector")
        search_results[subquery] = list(search_client.search(
            search_text=subquery,
            vector_queries=[vector_query],
            select=["title", "chunk", "chunk_id"],
            top=nr_search,
        ))

    #print(f"length search_results: {len(search_results)}")

    #Process results with or without RRF
    if rerank:
        #Using search_results, create two dictionaries.
        #result_dict contains the query as keys with chunk_id and score as values
        #chunk_dict contains all the info from search_results
        result_dict, chunk_dict = process_search_results(search_results)

        #rrf_scores is a dictionary containing the ranking of all the documents from all subqueries fused together.
        #It usually contains less documents than search_results has since it fuses together duplicate queries.
        #The documents are sorted in descending order with the highest similarity scoring documents at the beginning
        #Take the top nr_rrf_docs documents to give to the LLM 
        rrf_scores = reciprocal_rank_fusion(result_dict)
        #print(f"length rrf_scores: {len(rrf_scores)}")
        top_documents = [
            {
                "title": chunk_dict[chunk_id]["title"],
                "chunk": chunk_dict[chunk_id]["chunk"]
            }
            for chunk_id in rrf_scores.keys()
            if chunk_id in chunk_dict
        ][:nr_rrf_docs]
        #print(f"length top_documents: {len(top_documents)}")
    else:
        #Skip RRF and return top nr_docs documents directly
        top_documents = [
            {"title": result["title"], "chunk": result["chunk"]}
            for subquery_results in search_results.values()
            for result in subquery_results][:nr_docs]


    #Extract chunks of the top_documents to export to df and use for evaluation
    top_chunks = [doc["chunk"] for doc in top_documents]
    #print(f"Top chunks: {top_chunks}")
    
    #Format documents for LLM readability
    sources_formatted_complex = "=================\n".join(
        [f'TITLE: {doc["title"]}, CONTENT: {doc["chunk"]}' for doc in top_documents]
    )

    #Generate LLM response with the relevant documents
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": grounded_prompt_expansion.format(
                    original_query=query,
                    transformed_query=query_expansion,
                    sources=sources_formatted_complex,
                )
            }
        ],
        model=deployment_name
    )
    return response.choices[0].message.content, top_chunks


def search_multiple_with_optional_ranking(query, rerank, search_client, deployment_name):
    query_multiple_response = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": MULTIPLE_PROMPT.format(query=query, max_num_queries_multiple=max_num_queries_multiple)
            }
        ],
        model=deployment_name,
        response_format=MultipleQueryEvent
    )

    #Contains list of subqueries decomposed from the original user query
    query_decomposition = json.loads(query_multiple_response.choices[0].message.content)["queries"]
    subquery_results = []
    
    #Stores all top chunks from all subqueries to put into the dataframe
    top_chunks = []

    #Perform vector search and reranking separately for each subquery
    #Store all results for all subqueries with their top nr_docs reranked documents in subquery_results
    for sub_query in query_decomposition:
        vector_query = VectorizableTextQuery(text=sub_query, k_nearest_neighbors=nr_search, fields="text_vector")
        search_results = list(search_client.search(
            search_text=sub_query,
            vector_queries=[vector_query],
            select=["title", "chunk"],
            top=nr_search,
        ))

        if rerank:
            reranker_input = [[sub_query, document['chunk']] for document in search_results]
            scores = reranker.compute_score(reranker_input, normalize=True)
            ranked_documents = [
                {
                    "title": document["title"],
                    "chunk": document["chunk"],
                    "score": score,
                }
                for document, score in zip(search_results, scores)
            ]
            sorted_documents = sorted(ranked_documents, key=itemgetter("score"), reverse=True)[:nr_docs]
        else:
            #Skip reranking and return top nr_docs documents directly
            sorted_documents = search_results[:nr_docs]
        
        #Store chunks
        top_chunks.extend([doc["chunk"] for doc in sorted_documents])

        subquery_results.append({
            "sub_query": sub_query,
            "documents": sorted_documents
        })

        #Format each sub_query separately with its sources
    sources_formatted_multiple = "\n\n".join(
        f"SUB-QUERY: {result['sub_query']}\n=================\n" +
        "\n=================\n".join(
            f"TITLE: {doc['title']}, CONTENT: {doc['chunk']}"
            for doc in result["documents"]
        )
        for result in subquery_results
    )

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": grounded_prompt_decomposition.format(
                    original_query=query,
                    transformed_query=query_decomposition,
                    sources=sources_formatted_multiple,
                )
            }
        ],
        model=deployment_name
    )

    return response.choices[0].message.content, top_chunks

In [34]:
def search_non_agentic(query, rerank, search_client, deployment_name):
    '''
    Execute non agentic retrieval; direct embedding of the user query
    '''
    #Configure vectorized query
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=nr_search, fields="text_vector")

    #Perform vector search directly on the query
    #Search_results is an iterator object containing the relevant documents
    #Convert result to list to prevent exhausting the iterator
    search_results = list(search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["title", "chunk"],
        top=nr_search,
    ))

    top_documents = search_results[:nr_docs]

    #Extract chunks of the top_documents to export to df and use for evaluation
    top_chunks = [doc["chunk"] for doc in top_documents]

    #Format documents for LLM readability
    sources_formatted = "=================\n".join(
        [f'TITLE: {doc["title"]}, CONTENT: {doc["chunk"]}' for doc in top_documents]
    )

    #Generate LLM response using the relevant documents
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": grounded_prompt_non_agentic.format(
                    original_query=query,
                    sources=sources_formatted,
                )
            }
        ],
        model=deployment_name
    )
    return response.choices[0].message.content, top_chunks

In [39]:
def retrieve(query, search_client, deployment_name, use_agentic=True, use_reranking=True):
    """
    Retrieves documents based on the query, with configurable options.

    Args:
    - query (str): The user's query.
    - use_agentic (bool): Whether to use agentic retrieval (classification and query transformation).
    - use_reranking (bool): Whether to use reranking or reciprocal rank fusion in agentic retrieval.

    Returns:
    str: The final response with the relevant documents.
    """


    if not use_agentic:
        #Direct embedding search without classification or query transformation
        response, top_chunks = search_non_agentic(query, rerank=use_reranking, search_client=search_client, deployment_name=deployment_name)
        #In case search_conversational is called, return response with empty chunk list for the dataframe
        #print("No classification in non agentic retrieval")
        return response, top_chunks

    #If agentic retrieval is enabled, proceed with classification
    classification = classify_query(query, deployment_name)
    #Based on classification and reranking configuration, perform the corresponding query transformation and reranking method
    if classification == 1:
        response = search_conversational(query, deployment_name)
        response, top_chunks = response, []
    elif classification == 2:
        response, top_chunks = (
            search_simple_with_optional_ranking(query, rerank=use_reranking, search_client=search_client, deployment_name=deployment_name) 
            if use_reranking 
            else search_simple_with_optional_ranking(query, rerank=False, search_client=search_client, deployment_name=deployment_name)
        )
    elif classification == 3:
        response, top_chunks = (
            search_complex_with_optional_ranking(query, rerank=use_reranking, search_client=search_client, deployment_name=deployment_name)
            if use_reranking 
            else search_complex_with_optional_ranking(query, False, search_client=search_client, deployment_name=deployment_name)
        )
    elif classification == 4:
        response, top_chunks = (
            search_multiple_with_optional_ranking(query, rerank=use_reranking, search_client=search_client, deployment_name=deployment_name) 
            if use_reranking 
            else search_multiple_with_optional_ranking(query, False, search_client=search_client, deployment_name=deployment_name)
        )
    else:
        print("ERROR classification is not in range 1:4")
    return response, top_chunks

In [46]:
def evaluate_all_combinations2(df_questions, query_column, index_names, methods, llm_deployments, num_iterations=1):
    """
    Evaluates all combinations of search indexes and retrieval methods, and saves responses in a dataframe.

    Arguments:
    - df (Dataframe): Pandas dataframe that contains the user queries (questions).
    - query_column (str): The column name in the df that contains the queries.
    - index_names (list): List of index names from Azure AI Search to evaluate.
    - methods (list): List of retrieval methods to evaluate. Each method is a tuple: (method_name, use_agentic, rerank).
    - llm_deployments (list): List of LLM deployment names used for classification, transformation, and generation.
    - num_iterations (int): Number of times to run each query for each model configuration.

    Returns:
    Dataframe: The updated dataframe with responses for each combination of indexes and methods.
    """

    #Temp storage for responses, time, and document chunks to put into dataframe
    all_results = []

    for deployment_name in llm_deployments:
        for idx_name in index_names:  
            #Iterate over all index names

            #Recreate searchclient each time a new index is used
            search_client = SearchClient(
                endpoint=service_endpoint,
                index_name=idx_name,
                credential=AzureKeyCredential(key)
            )
        
            for method_name, use_agentic, use_reranking in methods:  
                
                #Iterate over all methods
                try:
                    df_results = pd.read_pickle(f"Backups/backup_{deployment_name}_{idx_name}_{method_name}")
                    print(f"Succesfully loaded: {idx_name} | Method: {method_name} | LLM: {deployment_name}")

                except Exception as err:
                    print(err)
                    print(f"Evaluating Index: {idx_name} | Method: {method_name} | LLM: {deployment_name}")                    

                    results = []
                    # For each query, generate responses for each iteration, and for each method configuration
                    for query in df_questions[query_column]:
                        for iteration in range(num_iterations):
                            start_time = time.time()

                            response, top_chunks = retrieve(query, search_client=search_client, use_agentic=use_agentic, use_reranking=use_reranking, deployment_name=deployment_name)

                            end_time = time.time()
                            
                            processing_time = end_time - start_time
                            
                            result = {
                                "deployment_name":deployment_name,
                                "idx_name":idx_name,
                                "method_name":method_name,
                                "use_agentic":use_agentic,
                                "use_reranking":methods,
                                "query":query,
                                "iteration":iteration,
                                "response":response,
                                "time":processing_time,
                                "results":top_chunks                            
                            }
                            results.append(result)

                            #Sleep to prevent exceeding rate limits
                            time.sleep(4)
                
                    df_results = pd.DataFrame(results)
                    df_results.to_pickle(f"Backups/backup_{deployment_name}_{idx_name}_{method_name}")
                all_results.append(df_results)

    #Update the dataframe with the results from all queries
    df = pd.concat(all_results)
    df.to_pickle("full_dataframe_pickle")
    return df

In [ ]:
df = pd.read_excel('Vragen HAFIR.xlsx')

In [ ]:
#df_llm_response = evaluate_all_combinations2(df_vragen, "Vraag", all_index_names, selected_methods, deployment_names, 3)

#### Run inference pipeline in batches

In [213]:
import pandas as pd
import time

def process_in_batches(df, query_column, batch_size, index_names, methods, llm_deployments, num_iterations=3):
    """
    Processes the input dataframe in smaller batches and save results incrementally.
    
    """
    for start_idx in range(0, len(df), batch_size):
        #Extract a single batch of queries
        batch_df = df.iloc[start_idx:start_idx + batch_size].copy()

        print(f"Processing batch {start_idx // batch_size + 1}...")
        try:
            #Process the batch through evaluate_all_combinations
            updated_df = evaluate_all_combinations2(
                batch_df,
                query_column=query_column,
                index_names=index_names,
                methods=methods,
                llm_deployments=llm_deployments,
                num_iterations=num_iterations
            )
            
            #save the batch results into a separate file
            batch_file_name = f"results_batch_{start_idx}.csv"
            updated_df.to_csv(batch_file_name, index=False)
            print(f"Batch {start_idx // batch_size + 1} saved successfully as '{batch_file_name}'!")
        
        except Exception as e:
            print(f"Error occurred in batch {start_idx // batch_size + 1}: {e}")


In [ ]:
process_in_batches(df, "Vraag", batch_size=2, index_names= all_index_names, methods = selected_methods, llm_deployments=deployment_names, num_iterations=3,)

In [177]:
#Small df For testing the pipeline:
#Create a temporary dataframe containing only the first question
df_first_two = df.tail(1).copy()

#Execute the retrieval for only the first question
#updated_df = retrieve_multiple_indexes_and_methods(df_first_two, "Vraag", selected_methods, all_index_names)
updated_df = evaluate_all_combinations2(df_first_two, "Vraag", all_index_names, selected_methods, deployment_names, 3)

updated_df.head()

Evaluating Index: idx-exhaustiveknn-custom-vectorizer-384 | Method: Non-agentic retrieval | LLM: gpt-4o
Evaluating Index: idx-exhaustiveknn-custom-vectorizer-384 | Method: Agentic retrieval without reranking | LLM: gpt-4o
Evaluating Index: idx-exhaustiveknn-custom-vectorizer-384 | Method: Agentic retrieval with reranking | LLM: gpt-4o
Evaluating Index: idx-hnsw-openai-vectorizer-text-embedding-3-large | Method: Non-agentic retrieval | LLM: gpt-4o
Evaluating Index: idx-hnsw-openai-vectorizer-text-embedding-3-large | Method: Agentic retrieval without reranking | LLM: gpt-4o
Evaluating Index: idx-hnsw-openai-vectorizer-text-embedding-3-large | Method: Agentic retrieval with reranking | LLM: gpt-4o
